In [11]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install google-cloud-bigquery

In [13]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BigqueryExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#Loading the Cookie
cookie_id_df = spark.read \
      .format("bigquery") \
      .load(" degroup11.group11dataset.cookie_ID_houses")

#Getting the mortgage threshold
price_threshold = cookie_id_df.first()["possible_mortgage_amount"]
print(price_threshold)

# Load data from BigQuery.
df_kafka = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.house_pricing_kafka")    

df_kafka=df_kafka.drop("window")
df_kafka.printSchema()

df_batch = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.available_houses_for_individual")  
df_batch.printSchema()

df_combined = df_batch.union(df_kafka)

df_combined=df_combined.dropDuplicates(["Address", "Price"])

df_combined = df_combined.orderBy("Price", ascending=False)
df_combined.show()

df_combined.write.format("bigquery").\
option('table', "degroup11.group11dataset.top_10_houses").\
option("temporaryGcsBucket", "temp_degroup11"). \
mode("overwrite").save()

414389


Py4JJavaError: An error occurred while calling o421.load.
: java.lang.RuntimeException: Table degroup11.group11dataset.house_pricing_kafka not found
	at scala.sys.package$.error(package.scala:30)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.$anonfun$createRelationInternal$1(BigQueryRelationProvider.scala:82)
	at scala.Option.getOrElse(Option.scala:189)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelationInternal(BigQueryRelationProvider.scala:82)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:346)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:229)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:211)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:211)
	at org.apache.spark

In [12]:
from google.cloud import bigquery

#Initialize the BigQuery client
client = bigquery.Client(project="degroup11")

#Delete temporary kafka table
client.delete_table("degroup11.group11dataset.house_pricing_kafka", not_found_ok=True)

#Delete temporary batch table
client.delete_table("degroup11.group11dataset.available_houses_for_individual", not_found_ok=True)

#Delete cookie as session is over
client.delete_table("degroup11.group11dataset.cookie_ID_houses", not_found_ok=True)